In [64]:
import requests
from getpass import getpass
from lib.scraper import WikiSearch
from lib.wikienv import WikiEnv
from lib.wrappers import HotPotQAWrapper, LoggingWrapper
from lib.bot_interface import Bot

API_KEY = getpass('Enter your OpenAI key: ')

In [69]:
# AGENT = 'teaLLM'
AGENT = 'ReAct'
# AGENT = 'CoT'

In [70]:
ENVS = {'teaLLM' : WikiSearch(api_key=API_KEY),
        'ReAct'  : WikiEnv(),
        'CoT'    : WikiEnv()} 
env = ENVS[AGENT]
env = HotPotQAWrapper(env, split='dev')
env = LoggingWrapper(env)
bot = Bot(api_key=API_KEY)

def step(env, action):
    attempts = 0
    while attempts < 10:
        try:
            return env.step(action)
        except requests.exceptions.Timeout:
            attempts += 1

In [71]:
folder = './prompts/'
prompt_file = folder + f'{AGENT}.txt'
with open(prompt_file, 'r') as f:
    webthink_prompt = '\n'.join(f.readlines())
    
def webthink(question=None, idx=None, bot=bot, prompt=webthink_prompt, to_print=True):
    if not (idx is None):
        question = env.reset(idx=idx)
    else:
        env.reset()
    if to_print and not idx is None: print(idx, question)
    prompt += question + "\n"
    if AGENT == 'CoT':
        steps = 0
        replies = ''
        while steps < 10:
            reply = bot(prompt)
            prompt += reply
            replies += reply
            if 'finish[' in replies.lower(): break
            steps += 1
        if to_print: print(replies)
        answer = ''
        if 'finish[' in replies.lower():
            answer = replies.lower().split('finish[')[-1][:-1]
        obs, r, done, info = env.step(f'finish[{answer}]')
        assert done
        if not (idx is None) and to_print: print(info, '\n')
        return r, info
    
    n_calls, n_badcalls = 0, 0
    for i in range(1, 8):
        n_calls += 1
        thought_action = bot(prompt + f"Thought {i}:", stop=[f"\nObservation {i}:"])
        try:
            thought, action = thought_action.strip().split(f"\nAction {i}: ")
        except:
            print('ohh...', thought_action)
            n_badcalls += 1
            n_calls += 1
            thought = thought_action.strip().split('\n')[0]
            action = bot(prompt + f"Thought {i}: {thought}\nAction {i}:", stop=[f"\n"]).strip()
            
        obs, r, done, info = step(env, action[0].lower() + action[1:])
        obs = obs.replace('\\n', '')
        step_str = f"Thought {i}: {thought}\nAction {i}: {action}\nObservation {i}: {obs}\n"
        prompt += step_str
        if to_print: print(step_str)
        if done: break
    if not done:
        finish_str = 'finish | NO ANSWER' if AGENT == 'teaLLM' else 'finish[NO ANSWER]'
        obs, r, done, info = step(env, finish_str)
        
    if not (idx is None) and to_print: print(info, '\n')
    info.update({'n_calls': n_calls, 'n_badcalls': n_badcalls, 'traj': prompt})
    return r, info

## Custom questions

In [72]:
# teamLLM should generally provide more comprehensive and informative answers
# answer: Ada Lovelace architecture, third-generation RT cores, fourth-generation Tensor Cores
_ = webthink(question='Why is the RTX 4090 so fast?',
             to_print=True)

Thought 1: I need to search for information on the RTX 4090 to understand why it is so fast.

Action 1: Search[RTX 4090]
Observation 1: The GeForce 40 series is the latest family of consumer-level graphics processing units developed by Nvidia, succeeding the GeForce 30 series. The series was announced on September 20, 2022, at the GPU Technology Conference (GTC) 2022 event.. The RTX 4090 was released as the first model of the series on October 12, 2022, launched for $1,599 US,[1] and the 16GB RTX 4080 was released on November 16, 2022 for $1,199 US. An RTX 4080 12GB was announced in September 2022, originally to be priced at $899 US, however following some controversy in the media it was "unlaunched" by Nvidia. On January 5, 2023, that model would be released as the RTX 4070 Ti for $799 US.

Thought 2: The passage does not directly explain why the RTX 4090 is so fast. I need to look up the specifications or reviews of the RTX 4090 to find the answer.

Action 2: Lookup[RTX 4090 speed]
O

## HotPotQA

In [63]:
import time, json
from tqdm import tqdm

idxs = list(range(7405))
rs = []
infos = []
old_time = time.time()
BEGIN = 0
END = 14
for i in tqdm(idxs[BEGIN:END]):
# for i in tqdm(idxs[758:1000]):
    print(f'INDEX = {i}')
    r, info = webthink(idx=i, to_print=True)
    rs.append(info['em'])
    infos.append(info)
    print(sum(rs), len(rs), sum(rs) / len(rs), (time.time() - old_time) / len(rs))
    print('-----------')
    print()

with open(f'results/{AGENT}_temp.json', 'w') as f:
    json.dump(infos, f)

  0%|          | 0/14 [00:00<?, ?it/s]

INDEX = 0
0 Question: Were Scott Derrickson and Ed Wood of the same nationality?
Thought 1: I need to find out the nationalities of Scott Derrickson and Ed Wood to see if they are the same.

Action 1: search | Scott Derrickson | nationality | nationality
Observation 1: There is no information related to nationality in the provided text. Scott Derrickson is an American filmmaker known for his work in horror films and the superhero film Doctor Strange.

Thought 2: Since there is no information about Scott Derrickson's nationality, I should search for Ed Wood's nationality and compare it to Scott Derrickson's.

Action 2: search | Ed Wood | nationality | nationality
Observation 2: There is no information related to nationality in the provided text. 

Summary: Edward Davis Wood Jr. was an American filmmaker, actor, screenwriter, and pulp novelist known for directing low-budget cult classic films in the 1950s. He had a fascination with the performing arts and pulp fiction from a young age, a

  7%|▋         | 1/14 [00:23<05:09, 23.79s/it]

Thought 7: Since I couldn't find direct information about the nationalities of Scott Derrickson and Ed Wood, I should try to infer their nationalities based on the information provided. Ed Wood is described as an American filmmaker, so I can conclude that he is American. Now, I need to search for Scott Derrickson's nationality to see if they are the same.

Action 7: search | Scott Derrickson | nationality | nationality
Observation 7: There is no information related to nationality in the provided text. Scott Derrickson is an American filmmaker known for his work in horror films and the superhero movie Doctor Strange.

{'steps': 8, 'answer': ' NO ANSWER', 'gt_answer': 'yes', 'question_idx': 0, 'reward': False, 'em': False, 'f1': 0} 

0 1 0.0 23.791435956954956
-----------

INDEX = 1
1 Question: What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?
Thought 1: I need to find out what government position the woman who portrayed Corliss Archer

 14%|█▍        | 2/14 [00:38<03:38, 18.20s/it]

...

Thought 7: "Blue Thunder" was co-written by Don Jakoby, but it does not star Nicolas Cage and Téa Leoni. I will continue searching through the similar articles to find a film that meets all the criteria.

Action 7: search | Double Team (film) | co-written by Don Jakoby? | co-written
Observation 7: There is no information in the text provided about whether the film "Double Team" was co-written by Don Jakoby. 

Double Team is a 1997 action comedy film directed by Tsui Hark and written by Don Jakoby and Paul Mones. It stars Jean-Claude Van Damme as counter-terrorist agent Jack Quinn, who is assigned to bring an elusive terrorist known as Stavros (Mickey Rourke) to justice. The film received negative reviews and was nominated for three Golden Raspberry Awards.

{'steps': 8, 'answer': ' NO ANSWER', 'gt_answer': 'David Weissman', 'question_idx': 11, 'reward': False, 'em': False, 'f1': 0} 

1 12 0.08333333333333333 20.807194749514263
-----------

INDEX = 12
12 Question: What year did Guns N R

 93%|█████████▎| 13/14 [04:39<00:26, 26.65s/it]

Thought 7: Since I couldn't find any information about Guns N Roses performing a promo for "Raw Deal" either, I should try to think of another movie starring Arnold Schwarzenegger as a former New York Police detective and search for that movie to find the year of the promo.

Thought 8: I should search for Arnold Schwarzenegger movies where he plays a former New York Police detective and look for any promos involving Guns N Roses.

Action 7: search | Arnold Schwarzenegger
Observation 7: Invalid input. Please try again with correct syntax.

{'steps': 8, 'answer': ' NO ANSWER', 'gt_answer': '1999', 'question_idx': 12, 'reward': False, 'em': False, 'f1': 0} 

1 13 0.07692307692307693 21.506892461043137
-----------

INDEX = 13
13 Question: Are Random House Tower and 888 7th Avenue both used for real estate?
Thought 1: I need to find out if Random House Tower and 888 7th Avenue are both used for real estate. I should start by searching for information on both buildings.

Action 1: search | R

100%|██████████| 14/14 [04:49<00:00, 20.71s/it]

Thought 3: I have found that Random House Tower is used for real estate, but there is no information about 888 7th Avenue being used for real estate. Since the question asked if both buildings are used for real estate, I cannot confirm for 888 7th Avenue based on the information available. I should finish the session.

Action 3: finish | Cannot confirm for 888 7th Avenue
Observation 3: Episode finished, reward = 0


{'steps': 3, 'answer': ' Cannot confirm for 888 7th Avenue', 'gt_answer': 'no', 'question_idx': 13, 'reward': False, 'em': False, 'f1': 0} 

1 14 0.07142857142857142 20.70925828388759
-----------

